Importación de las librerías python necesarias:

In [1]:
import sys
import math
import numpy as np
import pickle #modulo estandar para guardar y leer objetos en fichero
from sklearn import mixture #paquete para mixturas de Gaussianas


Lectura de parametros de la lınea de comandos:

In [2]:
if len(sys.argv)!=5:
  print('Usage: %s <trdata> <trlabels> <%%trper> <%%dvper>' % sys.argv[0]);
  sys.exit(1);
#X: recoge datos (sin etiquetas de clase)
X= np.load(sys.argv[1])['X'];
#xl: recoge las etiquetas de clase de los datos
xl=np.load(sys.argv[2])['xl'];
#trper: porcentaje de datos de entrenamiento (p.e. 80
trper=int(sys.argv[3]);
#dvper: porcentaje de datos de desarrollo (p.e. 20
dvper=int(sys.argv[4]);

Usage: c:\Users\david\anaconda3\envs\vsc\lib\site-packages\ipykernel_launcher.py <trdata> <trlabels> <%trper> <%dvper>


SystemExit: 1

c:\Users\david\anaconda3\envs\vsc\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Parametros para crear un modelo de mixturas de Gaussianas:

In [ ]:
K=1; #numero de componentes
rc=0.1; #factor de regularizacion de la matriz de covarianza
seed=23; #semilla para generacion de numeros aleatorio

Barajado y particion train-dev de los datos:

In [ ]:
N=X.shape[0];
np.random.seed(seed); perm=np.random.permutation(N);
X=X[perm]; xl=xl[perm];

# Selecting a subset for train and dev sets
Ntr=round(trper/100*N); #numero de datos de training
Xtr=X[:Ntr,:]; xltr=xl[:Ntr]; #datos de training
Ndv=round(dvper/100*N); #numero de datos de development
Xdv=X[N-Ndv:,:]; xldv=xl[N-Ndv:]; #datos de development

Etiquetas, numero de clases, etc:

In [ ]:
labs=np.unique(xltr).astype(int); #vector de etiquetas de clase sin repeticiones
C=labs.shape[0]; #numero de etiquetas de clase
N,D=Xtr.shape; #N=numero de datos de entrenamiento, D=dimension de los datos (numero de caracterısticas)
M=Xdv.shape[0]; #numero de datos de development
gtr=np.zeros((C,N)); #discriminantes de los datos de entrenamiento
gdv=np.zeros((C,M)); #discriminantes de los datos de development

Estandarizacion de los datos:

In [ ]:
# Normalise data
mu=np.mean(Xtr,axis=0); #media (de cada caracterıstica) del training
sigma=np.std(Xtr,axis=0); #desviacion tıpica (de cada caracterıstica) del training
sigma[sigma==0]=1;
Xtr=(Xtr-mu)/sigma; #training estandarizado
Xdv=(Xdv-mu)/sigma; #development estandarizado

Creacion, entrenamiento y evaluacion de una mixtura por clase:

In [ ]:
# Parameter estimation and soring samples
model=[] #modelo de clasificacion
for c,lab in enumerate(labs):
  Xtrc=Xtr[xltr==lab]; #datos de entrenamiento de la clase c
  Nc=Xtrc.shape[0]; #numero de datos de entrenamiento de la clase c
  pc=Nc/N; #prior de la clase c
  ##PROBAR COSES ACÍ
  gmm=mixture.GaussianMixture(n_components=K, reg_covar=rc, random_state=seed); #mixtura de K Gaussianas ajustada con Xtrc
  gmm.fit(Xtrc);
  #gtrygdvson las discriminantes de los datos Xtr y Xdv
  gtr[c]=math.log(pc)+gmm.score_samples(Xtr);
  gdv[c]=math.log(pc)+gmm.score_samples(Xdv);
  model.append((pc,gmm));

Clasificacion, calculo de error y almacenamiento del model:

In [ ]:
  # Classification of training and eval sets for error estimation
idx=np.argmax(gtr,axis=0);
etr=np.mean(np.not_equal(labs[idx],xltr))*100;
idx=np.argmax(gdv,axis=0);
edv=np.mean(np.not_equal(labs[idx],xldv))*100;

print('  K      rc   etr   edv')
print('--- ------- ----- -----')
print(f'{K:3} {rc:3.1e} {etr:5.2f} {edv:5.2f}');

filename = 'gmm.K1.rc0.1.mod'
pickle.dump(model, open(filename, 'wb'))
